# Trabajo integrador - Parte 1
## Python y Numpy

**Nombre**:

In [ ]:
import numpy as np

## Ejercicio 1

Dada una matriz en formato *numpy array*, donde cada fila de la matriz representa un vector matemático, se requiere computar las normas $l_0$, $l_1$, $l_2$, $l_{\infty}$, según la siguientes definiciones:

\begin{equation}
    ||\mathbf{x}||^{p} = \bigg(\sum_{j=1}^{n}{|x_i|^p}\bigg)^{\frac{1}{p}}
\end{equation}

con los casos especiales para $p=0$ y $p=\infty$ siendo:

\begin{equation}
    \begin{array}{rcl}
        ||\mathbf{x}||_0 & = & \bigg(\sum_{j=1 \wedge x_j != 0}{|x_i|}\bigg)\\
        ||\mathbf{x}||_{\infty} & = & \max_{i}{|x_i|}\\
    \end{array}
\end{equation}

In [2]:
import numpy as np

def norma_p(matriz,n_norma):
    if n_norma==0:
        norma=np.sum(matriz != 0, axis=1).sum()
    elif n_norma==np.inf:
        norma = np.max(np.sum(np.abs(matriz), axis=1))
    else:
        norma=np.sum(np.abs(matriz)**n_norma)**(1/n_norma)
    return norma
    print(norma)

matriz=np.array(np.random.randint(0,5,(4,4)))
print(matriz)

norma_p(matriz,0)




[[2 1 2 1]
 [4 3 1 1]
 [0 4 1 3]
 [4 0 0 1]]


13

## Ejercicio 2

En clasificación contamos con dos arreglos, la “verdad” y la “predicción”. Cada elemento de los arreglos pueden tomar dos valores, “True” (representado por 1) y “False” (representado por 0). Entonces podemos definir 4 variables:

* True Positive (TP): El valor verdadero es 1 y el valor predicho es 1
* True Negative (TN): El valor verdadero es 0 y el valor predicho es 0
* False Positive (FP): El valor verdadero es 0 y el valor predicho es 1
* False Negative (FN): El valor verdadero es 1 y el valor predicho es 0

A partir de esto definimos:

* Precision = TP / (TP + FP)
* Recall = TP / (TP + FN)
* Accuracy = (TP + TN) / (TP + TN + FP + FN)
 
Calcular las 3 métricas con Numpy y operaciones vectorizadas.

In [3]:
truth = np.array([1,1,0,1,1,1,0,0,0,1])
prediction = np.array([1,1,1,1,0,0,1,1,0,0])

print(truth)
print(prediction)

# Verdad y prediccion

tp=np.sum(truth*prediction) # np.sum([1 for t,p in zip(truth,prediction) if t==1 and p==1])
tn=np.count_nonzero((truth+prediction)==0) # np.sum([1 for t,p in zip(truth,prediction) if t==1 and p==0])
fp=np.sum([1 for t,p in zip(truth,prediction) if t==0 and p==1])
fn=np.sum([1 for t,p in zip(truth,prediction) if t==1 and p==0])

print(f"TP =",tp, "TN =",tn, "FP =",fp,"FN =", fn)

# Metricas

# Precision

precision=tp/np.sum(tp+fp)
print(f"Precision = TP/(TP+FP) = ",precision)

# Recall

recall=tp/np.sum(tp+fn)
print(f"Recall = TP/(TP + FN) =",recall)

# Accuracy

accuracy=np.sum(tp+tn)/np.sum(tp+tn+fp+fn)
print(f"Accuracy = (TP+TN)/(TP + TN + FP + FN) =",accuracy)







[1 1 0 1 1 1 0 0 0 1]
[1 1 1 1 0 0 1 1 0 0]
TP = 3 TN = 1 FP = 3 FN = 3
Precision = TP/(TP+FP) =  0.5
Recall = TP/(TP + FN) = 0.5
Accuracy = (TP+TN)/(TP + TN + FP + FN) = 0.4


## Ejercicio 3

Crear una función que separe los datos en train-validation-test. Debe recibir de parametros:

- X: Array o Dataframe que contiene los datos de entrada del sistema.
- y: Array o Dataframe que contiene la(s) variable(s) target del problema.
- train_percentage: _float_ el porcentaje de training.
- test_percentage: _float_ el porcentaje de testing.
- val_percentage: _float_ el porcentaje de validación.
- shuffle: _bool_ determina si el split debe hacerse de manera random o no.

Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [4]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer # Dataset de prueba para probar funcionalidad de función

def split(X_input,
          Y_input,
          train_size,
          val_size,
          test_size,
          random_state=42,
          shuffle=True):
    
    np.random.seed(random_state) # seteo semilla para reproducibilidad de los datos en este por default es 42

    size = sum((train_size,test_size,val_size)) # calculo si la sum % de las 3 particiones suman 1
    if size != 1: # valido condicion que la sum % de las 3 particiones sumen 1
        print (f"La suma de % de splits debe ser 100%:", size, "Por favor ingresa los valores nuevamente")
    else:
        if not isinstance(X_input, pd.DataFrame): # convierto a DataFrame en el caso que X_input no lo sea
            X_input = pd.DataFrame(X_input, columns = ["col_" + str(i) for i in range(X_input.shape[1])]) # genero dataframe X_input

        n_samples = X_input.shape[0] # determino numero de muestras 

        split_series = np.random.choice(a = ["train","test","validation"], p = [train_size, test_size, val_size], size = n_samples) # genera particiones aleatorias para train, test y validation en funcion de los tamaños de informados en train_size, val_size y test_size
        split_series = pd.Series(split_series) # convierte las particiones en una serie pandas
        
        X_train, X_test, X_validation = X_input.iloc[split_series[split_series == "train"].index,:], X_input.iloc[split_series[split_series == "test"].index,:], X_input.iloc[split_series[split_series == "validation"].index,:] # subdivicion de conjuntos X_train, X_test, X_validation 
        if not isinstance(Y_input, pd.DataFrame): #convierto a DataFrame en el caso que Y_input no lo sea
            Y_input = pd.DataFrame(Y_input, columns=["target"])

        y_train, y_test, y_validation = Y_input.iloc[split_series[split_series == "train"].index, :], Y_input.iloc[split_series[split_series == "test"].index, :], Y_input.iloc[split_series[split_series == "validation"].index, :] #subdivicion de conjuntos Y_train, Y_test, Y_validation 
    
        return X_train, X_test, X_validation, y_train, y_test, y_validation



cancer=load_breast_cancer()

X_cancer, y_cancer = cancer.data, cancer.target

X_train, X_test, X_validation, y_train, y_test, y_validation=split(X_cancer,y_cancer,train_size=0.70,val_size=0.15,test_size=0.15)

# Imprimir las dimensiones de los conjuntos divididos
print("Dimension de X_cancer",X_cancer.shape)
print("Dimension de y_cancer",y_cancer.shape)
print("Dimensiones de X_train_cancer:", X_train.shape)
print("Dimensiones de X_test_cancer:", X_test.shape)
print("Dimensiones de X_validation_cancer:", X_validation.shape)
print("Dimensiones de y_train_cancer:", y_train.shape)
print("Dimensiones de y_test_cancer:", y_test.shape)
print("Dimensiones de y_validation_cancer:", y_validation.shape)



Dimension de X_cancer (569, 30)
Dimension de y_cancer (569,)
Dimensiones de X_train_cancer: (389, 30)
Dimensiones de X_test_cancer: (83, 30)
Dimensiones de X_validation_cancer: (97, 30)
Dimensiones de y_train_cancer: (389, 1)
Dimensiones de y_test_cancer: (83, 1)
Dimensiones de y_validation_cancer: (97, 1)
